In [2]:
import boto3
import sagemaker
import time
import json

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()

default_bucket = sagemaker_session.default_bucket() # or use your own custom bucket name
base_job_prefix = "training-v1" # or define your own prefix

model_package_group_name2 = f"{base_job_prefix}-model-group2"  # Model name in model registry
pipeline_name2 = f"{base_job_prefix}-pipeline2"  # SageMaker Pipeline name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [7]:
base_job_prefix_ = "pipeline"
s3_raw_data = f's3://{default_bucket}/{base_job_prefix_}/full/data'

In [8]:
from pipeline import get_pipeline

pipeline2 = get_pipeline(
    region=region,
    role=role,
    default_bucket=default_bucket,
    model_package_group_name=model_package_group_name2,
    pipeline_name=pipeline_name2,
    base_job_prefix=base_job_prefix
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource


INFO:sagemaker.processing:Uploaded None to s3://sagemaker-ca-central-1-770158757194/training-v1-evaluation-2023-11-29-17-26-33-623/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-ca-central-1-770158757194/training-v1-evaluation-2023-11-29-17-26-33-623/source/runproc.sh
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [9]:
pipeline2.steps

[ProcessingStep(name='training-v1PreProcess', display_name=None, description=None, step_type=<StepTypeEnum.PROCESSING: 'Processing'>, depends_on=None),
 TrainingStep(name='BirdClassificationOnDemandTrain', display_name=None, description=None, step_type=<StepTypeEnum.TRAINING: 'Training'>, depends_on=None),
 ProcessingStep(name='BirdClassificationOnDemandEval', display_name=None, description=None, step_type=<StepTypeEnum.PROCESSING: 'Processing'>, depends_on=None),
 ConditionStep(name='BirdClassificationOnDemandCondition', display_name=None, description=None, step_type=<StepTypeEnum.CONDITION: 'Condition'>, depends_on=None)]

In [10]:
pipeline2.upsert(role_arn=role)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.processing:Uploaded None to s3://sagemaker-ca-central-1-770158757194/training-v1-pipeline2/code/99561f5d552a8f42dc63c3a145acd1b3/sourcedir.tar.gz


Using provided s3_resource
Using provided s3_resource


INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-ca-central-1-770158757194/training-v1-pipeline2/code/94064aa2845c61700dd4c437259a77cf/runproc.sh
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.processing:Uploaded None to s3://sagemaker-ca-central-1-770158757194/training-v1-pipeline2/code/99561f5d552a8f42dc63c3a145acd1b3/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-ca-central-1-770158757194/training-v1-pipeline2/code/94064aa2845c61700dd4c437259a77cf/runproc.sh


Using provided s3_resource
Using provided s3_resource


{'PipelineArn': 'arn:aws:sagemaker:ca-central-1:770158757194:pipeline/training-v1-pipeline2',
 'ResponseMetadata': {'RequestId': 'fb03d585-0aa2-4651-8770-1f7b44734b50',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fb03d585-0aa2-4651-8770-1f7b44734b50',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '92',
   'date': 'Wed, 29 Nov 2023 17:26:40 GMT'},
  'RetryAttempts': 0}}

In [11]:

execution = pipeline2.start(
    parameters=dict(
        InputDataUrl=s3_raw_data, # loaction of the raw data
        ProcessingInstanceCount=1,
#         ProcessingInstanceType="ml.m5.xlarge",
#         TrainingInstanceCount=1,
#         TrainingInstanceType="ml.c5.4xlarge",#"ml.p3.2xlarge",#
        ModelApprovalStatus="PendingManualApproval",
        AnnotationFileName="classes.txt",
        ClassSelection="13, 17, 35, 36"
    )
)